In [1]:
import datetime as dt
import panel as pn
import pandas as pd
import holoviews as hv
from holoviews import streams
import hvplot.pandas
import os
import warnings

#warnings.filterwarnings("ignore")
hv.extension("bokeh")
pn.extension("tabulator")

# Read in csv file
print(os.getcwd())
#data = pd.read_csv("test_files/test_one_step_hydrograph_pentad.csv")
data = pd.read_csv("../../internal_data/hydrograph_pentad_kghm.csv")
data = pd.read_csv("../../../../kyg_data_forecast_tools/intermediate_data/hydrograph_day.csv")
print(data)

/Users/bea/Documents/GitHub/SAPPHIRE_forecast_tools/apps/backend/tests
       index   code  count    mean    std    min      5%     25%    50%  \
0          0  15013   23.0   1.861  0.525   1.26   1.290   1.395   1.85   
1          1  15013   23.0   1.923  0.634   1.25   1.290   1.380   1.83   
2          2  15013   23.0   1.904  0.562   1.25   1.284   1.400   1.81   
3          3  15013   23.0   1.877  0.528   1.24   1.273   1.420   1.80   
4          4  15013   23.0   1.849  0.524   1.24   1.261   1.420   1.76   
...      ...    ...    ...     ...    ...    ...     ...     ...    ...   
22423  22625  17462   23.0  41.874  4.789  33.40  34.100  38.050  43.20   
22424  22626  17462   23.0  41.704  4.832  33.40  34.070  38.050  43.20   
22425  22627  17462   23.0  41.674  4.767  33.40  34.070  38.050  43.20   
22426  22628  17462   23.0  41.543  4.729  33.40  34.070  38.050  43.20   
22427  22629  17462   23.0  41.500  4.686  33.40  34.070  38.050  43.20   

          75%     95%    max

In [2]:
dataf = data[data['code']==16936]
dataf = data[data['code']==15194]
print(dataf)

      index   code  count   mean    std   min     5%    25%    50%    75%  \
5475   5475  15194   24.0  2.196  0.432  1.56  1.670  1.890  2.175  2.455   
5476   5476  15194   24.0  2.190  0.434  1.56  1.670  1.830  2.175  2.455   
5477   5477  15194   24.0  2.185  0.435  1.56  1.670  1.808  2.170  2.455   
5478   5478  15194   24.0  2.182  0.423  1.56  1.670  1.798  2.170  2.455   
5479   5479  15194   24.0  2.190  0.431  1.56  1.670  1.792  2.170  2.485   
...     ...    ...    ...    ...    ...   ...    ...    ...    ...    ...   
5835   5835  15194   23.0  2.273  0.467  1.56  1.685  1.950  2.200  2.550   
5836   5836  15194   23.0  2.249  0.471  1.56  1.685  1.850  2.170  2.550   
5837   5837  15194   23.0  2.262  0.472  1.68  1.685  1.850  2.170  2.570   
5838   5838  15194   23.0  2.246  0.464  1.56  1.685  1.900  2.170  2.510   
5839   5839  15194   23.0  2.228  0.468  1.56  1.685  1.850  2.150  2.470   

        95%   max  2025  2024        date  day_of_year  
5475  2.828  3.42 

In [6]:
# Use hvplot to plot the all columns against the date column in one figure
plotmean = dataf.hvplot(x="day_of_year", y=["mean"], kind="line", width=800, height=400)
plotmin = dataf.hvplot(x="day_of_year", y=["min"], kind="line", width=800, height=400)
plotmax = dataf.hvplot(x="day_of_year", y=["max"], kind="line", width=800, height=400)
plot05 = dataf.hvplot(x="day_of_year", y=["5%"], kind="line", width=800, height=400)
plot25 = dataf.hvplot(x="day_of_year", y=["25%"], kind="line", width=800, height=400)
plot50 = dataf.hvplot(x="day_of_year", y=["50%"], kind="line", width=800, height=400)
plot75 = dataf.hvplot(x="day_of_year", y=["75%"], kind="line", width=800, height=400)
plot95 = dataf.hvplot(x="day_of_year", y=["95%"], kind="line", width=800, height=400)
plot2024 = dataf.hvplot(x="day_of_year", y=["2024"], kind="line", width=800, height=400)
plot2025 = dataf.hvplot(x="day_of_year", y=["2025"], kind="line", width=800, height=400)

allplots = plot05 * plot25 * plotmean * plot50 * plot75 * plot95 * plotmin * plotmax * plot2024 * plot2025 
allplots

:Overlay
   .Curve.I    :Curve   [day_of_year]   (5%)
   .Curve.II   :Curve   [day_of_year]   (25%)
   .Curve.III  :Curve   [day_of_year]   (mean)
   .Curve.IV   :Curve   [day_of_year]   (50%)
   .Curve.V    :Curve   [day_of_year]   (75%)
   .Curve.VI   :Curve   [day_of_year]   (95%)
   .Curve.VII  :Curve   [day_of_year]   (min)
   .Curve.VIII :Curve   [day_of_year]   (max)
   .Curve.IX   :Curve   [day_of_year]   (2024)
   .Curve.X    :Curve   [day_of_year]   (2025)

In [7]:
# Show the legend in the hv plot
#p = hv.Curve(dataf, kdims='pentad', vdims=['2000']).opts(legend_position='top_left')

p = hv.Curve(dataf, kdims='pentad', vdims=['2000'], label='2000') *\
hv.Curve(dataf, kdims='pentad', vdims=['2001'], label='2001') *\
hv.Curve(dataf, kdims='pentad', vdims=['2002'], label='2002') *\
hv.Curve(dataf, kdims='pentad', vdims=['2003'], label='2003')
p.opts(width=600, legend_position='top_left')
p


DataError: Supplied data does not contain specified dimensions, the following dimensions were not found: ['pentad', '2000']

PandasInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html

# Experiment with interactive data table and scatter plot


In [16]:
df = data[["Code", "pentad", "2000", "2001"]]

station_widget = pn.widgets.Select(name='Station', options=data['Code'].unique().tolist(), value=15194)

def update_data(value):
    print("Widget\n", value)
    dfu = df[df['Code'] == value]
    return dfu

def update_scatter(value):
    dfp = update_data(value)
    print("Map dfu\n", dfp.head())
    scatter_plot = dfp.hvplot(kind="scatter", x="2000", y="2001", c="pentad").opts(title=f"{value}")
    return scatter_plot

scatter_plot = pn.panel(pn.bind(update_scatter, station_widget))

layout = pn.Column(
    station_widget,
    pn.Row(
        scatter_plot,
    ))

layout.servable()


Widget
 15194
Map dfu
       Code  pentad  2000  2001
288  15194       1  2.27  1.73
289  15194       2  2.27  1.73
290  15194       3  2.27  1.62
291  15194       4  1.94  1.50
292  15194       5  1.85  1.50


BokehModel(combine_events=True, render_bundle={'docs_json': {'a7221d8f-acd4-4078-bf61-0fdcf464cf84': {'version…

In [61]:
import panel as pn
import pandas as pd
import holoviews as hv
from holoviews import streams

pn.extension("tabulator")

df = pd.DataFrame(data={"c": ["a", "a", "b"], "x": [0, 5, 2], "y": [0, 3, 7]})

station = pn.widgets.Select(name='Station', options=df['c'].unique().tolist(), value='a')

def filter_df(df, value):
    return df[df['c'] == value]

scatter = hv.Scatter(filter_df(df, station.value), "x", "y").opts(xlim=(-10, 10),
                                        size=10,
                                        selected=[1],
                                        tools=['tap'])

dff = pn.bind(filter_df, station)

@pn.depends(station.param.value)
def update_table(value):
    ''' link scatter plot selection to table row '''
    print('station_selected', value)
    #dffi = filter_df(df, value)
    tabulator = pn.widgets.Tabulator(dff, selection=[], selectable='checkbox')
    return tabulator

def update_scatter(selection):
    ''' link table row selection to scatter plot '''
    print('row_selected', selection)
    return hv.Scatter(dff, "x", "y").opts(xlim=(-10, 10),
                                         ylim=(-10, 10),
                                         size=10,
                                         selected=selection,
                                         tools=['tap', 'box_select'])

def point_selected(index):
    ''' link scatter plot selection to table row '''
    print('point_selected', index)
    tabulator.selection = index


tabulator = update_table(station.value)
table = pn.panel(tabulator)
plot = hv.DynamicMap(pn.bind(update_scatter, tabulator.param.selection))

sel = streams.Selection1D(source=plot)

sel.param.watch_values(point_selected, 'index')
station.param.watch(update_table, 'value')
# tabulator.param.watch_values(row_selected, 'selection')

pn.Column(station,
          pn.Row(table, plot))#.servable()

station_selected a


TypeError: filter_df() missing 1 required positional argument: 'value'

In [2]:
from bokeh.models.widgets.tables import SelectEditor
df = pd.DataFrame([
    ['a', 'Hello', 0, 'keep'],
    ['b', 'Hi', 0, 'keep'],
    ['c', 'World', 0, 'keep'],
    ['d', 'Planet', 1, 'keep'],
    ['e', 'Earth', 1, 'keep']
], columns=['id', 'name', 'predicted', 'decision'])

# Create list of subsets of the data:
dfs = [df[df.predicted.eq(c)] for c in (0, 1)]

# Use the Select widget to switch between the subsets
select = pn.widgets.Select(name='Cluster #', value=0, options=list(range(len(dfs))))

# Create an interactive table, returning the subset based on the select widget:
def table_creator(cluster):
    return dfs[cluster]

interactive_table = pn.bind(table_creator, select)

# Use the interactive table as the data inside the Tabulator widget:
tabulator = pn.widgets.Tabulator(
    interactive_table,
    editors={'decision': SelectEditor(options=['keep', 'exclude']),
             'id': None, 'name': None, 'predicted': None}
)

# Display Select and Tabulator widgets side by side:
pn.Column(
    select,
    tabulator
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'adf4b3ff-4669-4ea0-93e3-3bfe1413c963': {'version…

In [ ]:

df = data[["Code", "2000", "2001"]]
station = pn.widgets.Select(name='Code', options=data['Code'].unique().tolist(), value=15194)
print("station: ", station.value)

@pn.depends(selected_code=station.param.value)
def update_table(selected_code):
    # Print datatypes in dfsel
    dfsel = df[df['Code']==selected_code]
    return dfsel

tabulator = pn.widgets.Tabulator(update_table, selection=[])

@pn.depends(selection=tabulator.param.selection, station=station.param.value)
def update_scatter(selection, station):
    print('station_selected', station)
    ''' link table row selection to scatter plot '''
    print('row_selected', selection)
    dfsel = update_table(station)
    axmin = min(min(dfsel['2000']), min(dfsel['2001']))
    axmax = max(max(dfsel['2000']), max(dfsel['2001']))
    return hv.Scatter(dfsel, "2000", "2001").opts(size=10,
                                         xlim=(axmin, axmax),
                                         ylim=(axmin, axmax),
                                         selected=selection,
                                         tools=['tap', 'box_select'])

#plot = pn.panel(update_scatter)

def point_selected(index):
    ''' link scatter plot selection to table row '''
    print('point_selected', index)
    tabulator.selection = index

plot = hv.DynamicMap(pn.bind(update_scatter, tabulator.param.selection, station))

sel = streams.Selection1D(source=plot)

sel.param.watch_values(point_selected, 'index')

@pn.depends(station.param.value)
def update_data(value):
    dfsel = update_table(value)
    tabulator.value = dfsel
    return dfsel

idf = df.interactive()

#plot.add_periodic_callback(lambda: point_selected(sel.index), 500)

# tabulator.param.watch_values(row_selected, 'selection')

pn.Row(station, idf, tabulator, plot)

station:  15194


AttributeError: 'function' object has no attribute 'owner'

: 